<a href="https://www.kaggle.com/code/aminshaker/practical-ml-learning-huggingface-project?scriptVersionId=180719270" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 1. Ignore warnings

In [1]:
import warnings

warnings.filterwarnings('ignore')

# 2. Import necessary libraries

In [2]:
import requests
import transformers
from bs4 import BeautifulSoup
from transformers import pipeline

2024-05-31 07:32:01.997518: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-31 07:32:01.997721: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-31 07:32:02.176482: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# 3. Extract article from medium

In [3]:
article = ''
article_url = 'https://medium.com/analytics-vidhya/openai-gpt-3-language-models-are-few-shot-learners-82531b3d3122'

# to avoid 403 error
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

request = requests.get(article_url, headers = headers)

if request.status_code == 200:
    soup = BeautifulSoup(request.content, 'html.parser')

    # Get only paragraphs of article
    all_paragraphs = soup.find_all('p', {'class' : "pw-post-body-paragraph"})
    article_content = [paragraph.text for paragraph in all_paragraphs]
    article = ''.join(article_content)
else:
    print("Please check article url and try again.")
    exit(0)

# 4. Chunk article content to process by summarizer model

In [4]:
max_chunk = 500

article = article.replace('.', '')
article = article.replace('?', '')
article = article.replace('!', '')

sentences = article.split(' ')
current_chunk = 0 
chunks = []

for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        chunks.append(sentence.split(' '))
        
for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

# 5. Generate summary of article

In [5]:
# Load summarizer model
summarizer = pipeline("summarization")

result = summarizer(chunks)
result[1]['summary_text'].strip()

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

'GPT-3 is upgraded with 175 billion parameters, tailors and escalates the Gpt-2 architecture . It is capable of performing machine translation, question-answering, reading conceptual tasks, scripting poems, and elementary math . It can even generate news articles in which human evaluators have difficulty distinguishing from articles written by humans .'